<a href="https://colab.research.google.com/github/germancruzram/GIS_PUNTO---SHP_POLILINEA/blob/main/shp_punto-shp_polilinea3D_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CONVIRTIENDO SHP PUNTOS -- SHP POLINEA 3D. GPS**

**ING. GERMAN CRUZ RAMIREZ🇳**

---



**1. INSTALANDO LIBRERIAS A UTILIZAR**

In [1]:
# Instalación de librerias
!pip install geopandas
!pip install shapely
!pip install folium

**2. CREANDO LAS CARPETAS DE TRABAJO**

In [9]:
import os


folders_to_create = ["/content/input_shp", "/content/output"]

for folder_path in folders_to_create:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Carpeta {folder_path} creada correctamente.")
    else:
        print(f"La carpeta {folder_path} ya existe.")


La carpeta /content/input_shp ya existe.
La carpeta /content/output ya existe.


**3. CONVIRTIENDO UN SHP DE PUNTOS A UN SHP POLI-LINEA**

Recuerde subir los archivos shp con los puntos a procesar

In [10]:
import os
import geopandas as gpd
from shapely.geometry import LineString
import zipfile

input_dir_path = "/content/input_shp"
output_dir_path = "/content/output"

if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

shp_files = [file for file in os.listdir(input_dir_path) if file.endswith('.shp')]

if not shp_files:
    raise FileNotFoundError("No se encontraron archivos SHP en el directorio de entrada.")

shp_file_name = os.path.splitext(shp_files[0])[0]

input_shp_path = os.path.join(input_dir_path, shp_files[0])
puntos_gdf = gpd.read_file(input_shp_path)

if 'Elevation' not in puntos_gdf.columns:
    raise ValueError("El GeoDataFrame no tiene la columna 'Elevation' necesaria.")

coords_z = [(row['Easting'], row['Northing'], row['Elevation']) for idx, row in puntos_gdf.iterrows()]

linea_z = LineString(coords_z)

linea_gdf = gpd.GeoDataFrame({'geometry': [linea_z], 'cod_tramo': [''] * 1})
linea_gdf.crs = puntos_gdf.crs

linea_gdf['cod_tramo'] = linea_gdf['cod_tramo'].astype(str).str.pad(width=12, side='right')

output_shp_path = os.path.join(output_dir_path, f"{shp_file_name}.shp")

linea_gdf.to_file(output_shp_path)

zip_filename = f"{shp_file_name}.zip"
zip_filepath = os.path.join(output_dir_path, zip_filename)

with zipfile.ZipFile(zip_filepath, 'w') as zipf:
    for f in os.listdir(output_dir_path):
        if f.startswith(shp_file_name) and not f.endswith('.zip'):
            zipf.write(os.path.join(output_dir_path, f), arcname=f)



**4. VIZUALIZAR SHP**

In [11]:
import os
import folium
import geopandas as gpd
from shapely.geometry import LineString

output_dir_path = "/content/output"

shp_files = [file for file in os.listdir(output_dir_path) if file.endswith('.shp')]
if not shp_files:
    raise FileNotFoundError("No se encontraron archivos SHP en la carpeta '/content/output/'.")

shp_file_path = os.path.join(output_dir_path, shp_files[0])
gdf = gpd.read_file(shp_file_path)

gdf.set_crs(epsg=32616, inplace=True)

def convert_2d(geom):
    if geom.has_z:
        return LineString([(x, y) for x, y, z in geom.coords])
    else:
        return geom

gdf['geometry'] = gdf['geometry'].apply(convert_2d)

# Convertir el GeoDataFrame al CRS WGS 84 (EPSG:4326) para usar con Google Maps
gdf_wgs84 = gdf.to_crs(epsg=4326)

# Calcular el centroide del conjunto de geometrías para centrar el mapa
centroid = gdf_wgs84.unary_union.centroid
avg_lat = centroid.y
avg_lon = centroid.x

# Crear una instancia de mapa de Folium en la ubicación del centroide
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=14)

# Añadir las geometrías del GeoDataFrame al mapa
folium.GeoJson(gdf_wgs84, name='geojson').add_to(m)

# Mostrar el mapa
m


